<a href="https://colab.research.google.com/github/mehrshad-sdtn/DeepLearning/blob/master/PyTorch/0_Pytorch_Basic_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: import all the necessary packages for common pytorch programs

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
# Create Fully-Connected network
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


#model = NN(784, 10)
#x = torch.randn(64, 784)
#print(model(x).shape)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 5

In [ ]:
# load Data
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# initialize network
model = NN(input_size= input_size, num_classes= num_classes).to(device)

In [ ]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# check accurcy on training and test
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data:')
  else:
    print('Checking accuracy on test data')

  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device= device)
      y = y.to(device= device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x) # 64, 10
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f" {float(num_correct)/float(num_samples)*100:.2f}%")
    model.train()


# train
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # data shaping
    data = data.to(device)
    targets = targets.to(device)
    data = data.reshape(data.shape[0], -1)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent
    optimizer.step()

  print(f"Epoch {epoch+1}/{num_epochs}: loss {loss}")
  check_accuracy(train_loader, model)
  check_accuracy(test_loader, model)







Epoch 1/5: loss 0.48897090554237366
Checking accuracy on training data:
 86.67%
Checking accuracy on test data
 87.31%
Epoch 2/5: loss 0.5054873824119568
Checking accuracy on training data:
 89.59%
Checking accuracy on test data
 90.14%
Epoch 3/5: loss 0.38287678360939026
Checking accuracy on training data:
 90.70%
Checking accuracy on test data
 91.21%
Epoch 4/5: loss 0.2688435912132263
Checking accuracy on training data:
 91.41%
Checking accuracy on test data
 91.79%
Epoch 5/5: loss 0.44690120220184326
Checking accuracy on training data:
 91.97%
Checking accuracy on test data
 92.28%
